# 1. Import

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import collections
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import sys
import os
import pickle

# 2. Bài toán và vấn đề mô hình hóa dữ liệu

### 2.1 Đặt vấn đề

**Bài toán dự đoán khả năng xảy ra loại tội phạm trộm cướp** bằng mô hình dữ liệu là một bài toán phân loại, trong đó mục tiêu là dự đoán liệu một địa điểm cụ thể có xảy ra tội phạm trộm cướp trong một khoảng thời gian nhất định hay không. 

Giải quyết vấn đề này giúp hỗ trợ các cơ quan thực thi pháp luật trong việc **điều phối lực lượng và ngăn chặn tội phạm**, cũng như cảnh báo về người dân và du khách nên có các biện pháp **phòng vệ cũng như nâng cao cảnh giác về tài sản cá nhân** trước các hiện tượng xã hội nguy hiểm.

Bằng cách dự đoán được những khu vực có nguy cơ cao xảy ra tội phạm, các cơ quan chức năng có thể tăng cường tuần tra, kiểm soát, từ đó giảm thiểu khả năng xảy ra tội phạm.

### 2.2 Phương pháp phân loại (classification)

Để giải quyết bài toán này, các mô hình học máy thường được sử dụng:

- Mô hình phân loại (classification): Mô hình được sử dụng để phân loại các điểm dữ liệu thành hai lớp, trong đó một lớp là khu vực có nguy cơ cao xảy ra tội phạm và lớp còn lại là khu vực có nguy cơ thấp hơn.
- Mô hình dự đoán điểm số: Mô hình dự đoán một điểm số cho mỗi điểm dữ liệu, trong đó điểm số càng cao thì khả năng xảy ra tội phạm lớn hơn.

Để xây dựng các mô hình, cần thu thập dữ liệu bao gồm các thông tin về địa điểm, thời gian, loại hình phạm tội, các yếu tố kinh tế - xã hội khác,... Các thông tin này được lấy từ các nguồn như dữ liệu lịch sử tội phạm, dữ liệu thống kê, dữ liệu từ các cảm biến,...

Sau khi xây dựng mô hình, mô hình cần được đánh giá để xác định độ chính xác của mô hình. Các siêu tham số của mô hình xác định bằng cách sử dụng các kỹ thuật khoa học dữ liệu (ví dụ: validation set, imbalance data hoặc cross-validation) và báo cáo quy trình tinh chỉnh mô hình (fine-tuning process).

Độ chính xác của mô hình càng cao thì khả năng dự đoán chính xác của mô hình càng cao. Hiệu suất của các mô hình phải được đánh giá bằng cách sử dụng các metric cho bài toán phân lớp (precision, accuracy, và recall) 

**Ở vấn đề của bài toán trên, sử dụng các mô hình phân loại sẽ có ưu điểm hơn:**
- Dễ hiểu và sử dụng: Mô hình classification chỉ đưa ra hai kết quả, đó là khu vực có nguy cơ cao xảy ra tội phạm hoặc khu vực có nguy cơ thấp xảy ra tội phạm. Giúp cho các cơ quan chức năng dễ dàng hiểu và sử dụng kết quả của mô hình.

- Dễ dàng áp dụng trực tiếp để xác định các khu vực có nguy cơ cao xảy ra tội phạm, có thể nhanh chóng triển khai các biện pháp phòng ngừa và đấu tranh phòng chống tội phạm.

### 2.3 Dữ liệu

In [2]:
data = pd.read_csv("Cleaned_Crimes.csv")
data

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Updated On
0,12552399,JE459045,2021-11-28 04:39:00,048XX W NELSON ST,051A,ASSAULT,AGGRAVATED - HANDGUN,STREET,False,False,2521,25,31,19,04A,2021,2021-12-05 15:42:51
1,12766744,JF324452,2022-07-20 05:30:00,062XX N KILDARE AVE,0820,THEFT,$500 AND UNDER,STREET,False,False,1711,17,39,12,06,2022,2023-01-03 15:40:27
2,12259489,JE101102,2021-01-02 10:00:00,016XX E 95TH ST,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,431,4,7,51,07,2021,2021-01-16 15:39:23
3,12398408,JE270791,2021-06-18 16:09:00,095XX S WOODLAWN AVE,0630,BURGLARY,ATTEMPT FORCIBLE ENTRY,RESIDENCE,False,False,511,5,8,50,05,2021,2021-06-25 15:40:42
4,12394621,JE265470,2021-06-12 02:30:00,006XX N DEARBORN ST,0810,THEFT,OVER $500,RESTAURANT,False,False,1832,18,42,8,06,2021,2021-06-19 15:40:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,12923495,JF510211,2022-12-10 23:00:00,044XX N LAKE SHORE DR SB,0820,THEFT,$500 AND UNDER,STREET,False,False,1914,19,46,3,06,2022,2023-01-03 15:40:27
2996,12572567,JE483269,2021-12-20 01:38:00,043XX S CAMPBELL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,922,9,15,58,08B,2021,2021-12-27 15:41:40
2997,12732281,JF282524,2022-06-16 19:22:00,027XX S DEARBORN ST,0460,BATTERY,SIMPLE,CHA PARKING LOT / GROUNDS,False,False,133,1,3,35,08B,2022,2023-01-03 15:40:27
2998,12888941,JF469040,2022-02-02 09:00:00,030XX S MICHIGAN AVE,0890,THEFT,FROM BUILDING,APARTMENT,False,False,133,1,3,35,06,2022,2023-01-03 15:40:27


# 3. Tiền xử lý dữ liệu trước khi mô hình hóa chúng

### 3.1 Thông tin về các yếu tô (cột) dữ liệu

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    3000 non-null   int64 
 1   Case Number           3000 non-null   object
 2   Date                  3000 non-null   object
 3   Block                 3000 non-null   object
 4   IUCR                  3000 non-null   object
 5   Primary Type          3000 non-null   object
 6   Description           3000 non-null   object
 7   Location Description  2986 non-null   object
 8   Arrest                3000 non-null   bool  
 9   Domestic              3000 non-null   bool  
 10  Beat                  3000 non-null   int64 
 11  District              3000 non-null   int64 
 12  Ward                  3000 non-null   int64 
 13  Community Area        3000 non-null   int64 
 14  FBI Code              3000 non-null   object
 15  Year                  3000 non-null   

In [4]:
cat_col_info_gas = data.select_dtypes (include = ['object', 'int64'])
cat_col_info_gas_df = pd.DataFrame(columns= cat_col_info_gas.columns)

for column in cat_col_info_gas.columns:
    frequencies = collections.Counter(cat_col_info_gas[column].to_list())
    print(column, ": numbers of unique values: ", len(frequencies.keys()), ", including: ", frequencies)
    cat_col_info_gas_df[column] = list([len(frequencies.keys()), frequencies])

cat_col_info_gas_df.index = ['num_diff_vals', 'distribution']
cat_col_info_gas_df

ID : numbers of unique values:  3000 , including:  Counter({12552399: 1, 12766744: 1, 12259489: 1, 12398408: 1, 12394621: 1, 12280449: 1, 12752107: 1, 12632624: 1, 12232479: 1, 12117365: 1, 12660507: 1, 12699748: 1, 12948672: 1, 12228386: 1, 12231114: 1, 12682460: 1, 12772175: 1, 12537289: 1, 12657901: 1, 12800338: 1, 12388251: 1, 12628610: 1, 12288332: 1, 12849311: 1, 12250414: 1, 12764254: 1, 12808397: 1, 12659958: 1, 12583770: 1, 12466609: 1, 12111852: 1, 12636520: 1, 12217673: 1, 12520097: 1, 12880396: 1, 12347426: 1, 12870151: 1, 12780764: 1, 12615658: 1, 12067535: 1, 12494584: 1, 12822164: 1, 12633294: 1, 12946067: 1, 12790902: 1, 12783111: 1, 12502433: 1, 12405550: 1, 12722547: 1, 12813494: 1, 12522029: 1, 12516869: 1, 12204210: 1, 12752134: 1, 12931870: 1, 12573875: 1, 12915886: 1, 12298182: 1, 12839734: 1, 12282507: 1, 12023356: 1, 12613864: 1, 12775106: 1, 12642491: 1, 12502566: 1, 12440178: 1, 12246251: 1, 12529542: 1, 12461094: 1, 12176168: 1, 12778360: 1, 12326776: 1, 1290

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Beat,District,Ward,Community Area,FBI Code,Year,Updated On
num_diff_vals,3000,3000,2961,2622,167,23,160,80,273,23,50,77,23,3,672
distribution,"{12552399: 1, 12766744: 1, 12259489: 1, 123984...","{'JE459045': 1, 'JF324452': 1, 'JE101102': 1, ...","{'2021-11-28 04:39:00': 1, '2022-07-20 05:30:0...","{'048XX W NELSON ST': 2, '062XX N KILDARE AVE'...","{'051A': 72, '0820': 221, '0910': 182, '0630':...","{'ASSAULT': 289, 'THEFT': 637, 'MOTOR VEHICLE ...","{'AGGRAVATED - HANDGUN': 106, '$500 AND UNDER'...","{'STREET': 777, 'PARKING LOT / GARAGE (NON RES...","{2521: 15, 1711: 7, 431: 13, 511: 14, 1832: 8,...","{25: 153, 17: 87, 4: 194, 5: 139, 18: 165, 2: ...","{31: 37, 39: 43, 7: 107, 8: 96, 42: 134, 37: 7...","{19: 56, 12: 10, 51: 19, 50: 10, 8: 134, 25: 1...","{'04A': 99, '06': 637, '07': 208, '05': 103, '...","{2021: 1147, 2022: 1402, 2020: 451}","{'2021-12-05 15:42:51': 4, '2023-01-03 15:40:2..."


### 3.2 Chọn tập dữ liệu phù hợp

- Về các yếu tố xảy ra tội phạm, đáng quan tâm hơn cả là các yếu tố Ngày tháng, Địa điểm, Quận đã từng diễn ra Các loại hình tội phạm

In [5]:
crime_predict_df = data.loc[:, ["Date", "Location Description", "Ward", "Primary Type"]]
crime_predict_df

,Date,Location Description,Ward,Primary Type
0,2021-11-28 04:39:00,STREET,31,ASSAULT
1,2022-07-20 05:30:00,STREET,39,THEFT
2,2021-01-02 10:00:00,PARKING LOT / GARAGE (NON RESIDENTIAL),7,MOTOR VEHICLE THEFT
3,2021-06-18 16:09:00,RESIDENCE,8,BURGLARY
4,2021-06-12 02:30:00,RESTAURANT,42,THEFT
...,...,...,...,...
2995,2022-12-10 23:00:00,STREET,46,THEFT
2996,2021-12-20 01:38:00,RESIDENCE,15,BATTERY
2997,2022-06-16 19:22:00,CHA PARKING LOT / GROUNDS,3,BATTERY
2998,2022-02-02 09:00:00,APARTMENT,3,THEFT


- Xử lý cột dữ liệu `Date`

In [6]:
crime_predict_df["Date"] = pd.to_datetime(crime_predict_df["Date"])

- Lựa chọn tháng (trong năm) mà loại tội phạm đã diễn ra, đồng thời xem xét hành vi phạm pháp diễn ra vào ban ngày hay ban đêm.

In [7]:
crime_predict_df["Month"] = crime_predict_df["Date"].dt.month
crime_predict_df["Session_day"] = crime_predict_df["Date"].dt.hour.apply(lambda x: "Day" if x >= 6 and x < 18 else "Night")
crime_predict_df

,Date,Location Description,Ward,Primary Type,Month,Session_day
0,2021-11-28 04:39:00,STREET,31,ASSAULT,11,Night
1,2022-07-20 05:30:00,STREET,39,THEFT,7,Night
2,2021-01-02 10:00:00,PARKING LOT / GARAGE (NON RESIDENTIAL),7,MOTOR VEHICLE THEFT,1,Day
3,2021-06-18 16:09:00,RESIDENCE,8,BURGLARY,6,Day
4,2021-06-12 02:30:00,RESTAURANT,42,THEFT,6,Night
...,...,...,...,...,...,...
2995,2022-12-10 23:00:00,STREET,46,THEFT,12,Night
2996,2021-12-20 01:38:00,RESIDENCE,15,BATTERY,12,Night
2997,2022-06-16 19:22:00,CHA PARKING LOT / GROUNDS,3,BATTERY,6,Night
2998,2022-02-02 09:00:00,APARTMENT,3,THEFT,2,Day


- Tinh gọn tối đa bộ dữ liệu

In [8]:
crime_predict_df = crime_predict_df.drop('Date', axis=1)

In [9]:
crime_predict_df

,Location Description,Ward,Primary Type,Month,Session_day
0,STREET,31,ASSAULT,11,Night
1,STREET,39,THEFT,7,Night
2,PARKING LOT / GARAGE (NON RESIDENTIAL),7,MOTOR VEHICLE THEFT,1,Day
3,RESIDENCE,8,BURGLARY,6,Day
4,RESTAURANT,42,THEFT,6,Night
...,...,...,...,...,...
2995,STREET,46,THEFT,12,Night
2996,RESIDENCE,15,BATTERY,12,Night
2997,CHA PARKING LOT / GROUNDS,3,BATTERY,6,Night
2998,APARTMENT,3,THEFT,2,Day


- Cuối cùng, chọn lọc các loại hình tội phạm liên quan về trộm cướp và đánh giá điểm dữ liệu (record về vụ phạm tội) là HIGH. Nếu không phải, thì đánh giá LOW.

Lưu vào dataframe `theft_predict_df` chỉ tội phạm trộm cướp.

In [10]:
# Tạo DataFrame
theft_predict_df = pd.DataFrame({
    "col1": crime_predict_df['Location Description'],
    "col2": crime_predict_df['Ward'], 
    "col3": crime_predict_df['Month'], 
    "col4": crime_predict_df['Session_day'], 
    'col5': crime_predict_df['Primary Type'].apply(lambda x: "HIGH" if x in ['THEFT', 'MOTOR VEHICLE THEFT', 'ROBBERY', 'BURGLARY'] else "LOW")
    
})

# 4. Xây dựng mô hình dữ liệu và đánh giá

### 4.1 Các thuật toán học không giám sát (unsupervised-learning algorithms)

  **Support vector machine (SVM)** là thuật toán với phương pháp học không giám sát được sử dụng để phân tách các đối tượng thành hai hoặc nhiều lớp,hoạt động bằng cách tìm một đường phân cách giữa các lớp sao cho các điểm của cùng một lớp nằm ở cùng một bên của đường phân cách. Đường phân cách này được gọi là đường phân cách tối ưu, và nó có độ dài ngắn nhất có thể.



### 4.2 Các thuật toán học có giám sát (supervised learning algorithms)

- **Random forest** là thuật toán phân loại dựa trên tập hợp các cây quyết định. Nó tạo ra nhiều cây quyết định bằng cách sử dụng các tập dữ liệu huấn luyện khác nhau. Mỗi cây quyết định được tạo ra bằng cách sử dụng một tập dữ liệu huấn luyện ngẫu nhiên, trong đó một số thuộc tính của dữ liệu đã bị loại bỏ. Mô hình random forest dự đoán lớp của một đối tượng bằng cách lấy số phiếu của từng cây quyết định.

- **Gradient Boost** là một thuật toán học tăng cường (ensemble learning) sử dụng để phân loại các đối tượng. Gradient Boost hoạt động bằng cách tạo ra một tập hợp các mô hình dự đoán. Mỗi mô hình dự đoán được tạo ra để giảm lỗi của mô hình trước đó. Mô hình Gradient Boost dự đoán lớp của một đối tượng bằng cách lấy tổng các dự đoán của từng mô hình.  Cụ thể, Gradient Boost sử dụng phương pháp gradient descent để tìm các tham số của mô hình dự đoán mới sao cho lỗi của mô hình tổng thể giảm đi.

- **XGBoost** là một biến thể của Gradient Boost. Nó được cải tiến để hoạt động hiệu quả hơn trên dữ liệu lớn. XGBoost sử dụng một số kỹ thuật để cải thiện hiệu suất như:
   + Cách tính toán lỗi: sử dụng một cách tính toán lỗi khác với Gradient Boost, cụ thể là được thiết kế để hoạt động hiệu quả hơn trên dữ liệu lớn.
   + Cách điều chỉnh trọng số: khác với Gradient Boost nên sẽ hoạt động hiệu quả hơn trên dữ liệu lớn.
   + Các tính năng nâng cao: XGBoost có một số tính năng nâng cao không có trong Gradient Boost: Regularization là một kỹ thuật giúp ngăn chặn mô hình bị quá khớp, Tree pruning là một kỹ thuật giúp loại bỏ các nút không cần thiết trong cây quyết định.

- **LightGBM** là một thuật toán boost khác cũng phổ biến và hiệu quả. Nó được thiết kế để hoạt động hiệu quả hơn trên dữ liệu phân loại, sử dụng một số kỹ thuật để cải thiện hiệu suất về cách tính toán lỗi và cách điều chỉnh trọng số. Ngoài ra LGBM còn có các tính năng nâng cao mà không có trong Gradient Boost và XGBoost:
    + Gradient boosting decision tree (GBDT) là một kỹ thuật boost được sử dụng trong LGBM, được thiết kế để hoạt động hiệu quả hơn trên dữ liệu phân loại.
    + CatBoost là một kỹ thuật boost được sử dụng, được thiết kế đặc biệt cho dữ liệu phân loại có thể được phân loại thành các danh mục.

### 4.3 Cross validation và fine-tuning processing

- **Cross-validation** là một kỹ thuật đánh giá hiệu suất của mô hình học máy. Nó chia tập dữ liệu thành các tập con, sau đó sử dụng một tập con để đào tạo mô hình và các tập con còn lại để đánh giá hiệu suất của mô hình. Có nhiều loại cross-validation khác nhau, nhưng phổ biến nhất là k-fold cross-validation. trong đó, tập dữ liệu được chia thành k tập con bằng nhau, mô hình được đào tạo trên k-1 tập con và đánh giá trên tập con còn lại, quá trình này được lặp lại k lần, với mỗi tập con được sử dụng làm tập dữ liệu đánh giá một lần. Một số lợi ích của cross-validation:
  + Đánh giá hiệu suất của mô hình một cách chính xác hơn trên dữ liệu thực tế, thay vì chỉ trên tập dữ liệu đào tạo, giúp đảm bảo rằng mô hình không bị quá khớp (overfit) với tập dữ liệu đào tạo.
  + Lựa chọn các tham số tối ưu cho mô hình, chẳng hạn như số lượng lớp học, số lượng cây quyết định, hoặc độ sâu của cây quyết định.
  + Kiểm tra độ ổn định của mô hình, nếu hiệu suất của mô hình thay đổi đáng kể khi sử dụng các tập dữ liệu đánh giá khác nhau, thì mô hình có thể không ổn định.

- **Fine-tuning processing** là quá trình tinh chỉnh các tham số của một mô hình đã được đào tạo trước. Nó có tác dụng cải thiện hiệu suất của mô hình trên một tập dữ liệu cụ thể. Backpropagation là một kỹ thuật học máy sử dụng đạo hàm để cập nhật các tham số của mô hình. Trong fine-tuning, backpropagation được sử dụng để cập nhật các tham số của mô hình sao cho mô hình có hiệu suất tốt hơn trên tập dữ liệu cụ thể. Một số lợi ích của fine-tuning:
  + Giúp tiết kiệm thời gian và tài nguyên bằng cách sử dụng một mô hình đã được đào tạo trước.
  + Dễ dàng thực hiện bằng cách sử dụng nhiều thư viện và công cụ học máy phổ biến.

### 4.4 Hiệu suất của mô hình

Trong bài toán phân lớp, các metric thường được sử dụng để đánh giá hiệu suất của mô hình bao gồm:
  - **Precision** là tỷ lệ giữa các dự đoán dương thực tế được phân loại chính xác với tổng số dự đoán dương.

                  Precision = True Positive / (True Positive + False Positive)
  - **Recall** là tỷ lệ giữa các dự đoán dương thực tế được phân loại chính xác với tổng số điểm dữ liệu thực sự thuộc lớp dương.

                  Recall = True Positive / (True Positive + False Negative)
  - **Accuracy** là tỷ lệ giữa tổng số dự đoán chính xác với tổng số điểm dữ liệu.

        Accuracy = (True Positive + True Negative) / (True Positive + True Negative + False Positive + False Negative)

Precision và recall là hai metric quan trọng trong bài toán phân lớp. Precision đo lường độ chính xác của các dự đoán dương, trong khi recall đo lường độ đầy đủ của các dự đoán dương

### 4.5 Áp dụng các thuật toán và kỹ thuật Machine Learning trên để mô hình hóa dữ liệu

**Random Forest**

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = theft_predict_df.copy()
# Chuyển đổi các cột categorical thành dạng số
for i in range(1, 5):
    df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test_RF, y_train, y_test_RF = train_test_split(df[[f"col{i}" for i in range(1, 5)]], df['col5'], test_size=0.25)

# Tạo mô hình Random Forest
model = RandomForestClassifier(n_estimators=1000, max_depth=5)

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred_RF = model.predict(X_test_RF)
# Đánh giá mô hình
print("=========================Đánh giá mô hình Random Forest===========================")
print(classification_report(y_test_RF, y_pred_RF))
print("Độ chính xác:", model.score(X_test_RF, y_test_RF))

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\1709781892.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\1709781892.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1


=========================Đánh giá mô hình Random Forest===========================
              precision    recall  f1-score   support

        HIGH       0.58      0.21      0.31       250
         LOW       0.70      0.92      0.80       500

    accuracy                           0.69       750
   macro avg       0.64      0.57      0.55       750
weighted avg       0.66      0.69      0.63       750

Độ chính xác: 0.6853333333333333


**Support Vector Machine**

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

df = theft_predict_df.copy()
for i in range(1, 5):
    df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
X_train, X_test_SVM, y_train, y_test_SVM = train_test_split(
    df[df.columns[:4]], df[df.columns[4]], test_size=0.25
)

# Tạo mô hình SVM
model_SVM = SVC(kernel="rbf", gamma="auto")


model_SVM.fit(X_train, y_train)

# Dự đoán các điểm dữ liệu trong tập kiểm tra
y_pred_SVM = model_SVM.predict(X_test_SVM)
print("=========================Đánh giá mô hình Support Vector Machine===========================")
print(classification_report(y_test_SVM, y_pred_SVM))
print("Độ chính xác:", model_SVM.score(X_test_SVM, y_test_SVM))


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\2738052359.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\2738052359.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1


=========================Đánh giá mô hình Support Vector Machine===========================
              precision    recall  f1-score   support

        HIGH       0.55      0.28      0.37       274
         LOW       0.68      0.87      0.76       476

    accuracy                           0.65       750
   macro avg       0.61      0.57      0.56       750
weighted avg       0.63      0.65      0.62       750

Độ chính xác: 0.652


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

df = theft_predict_df.copy()
for i in range(1, 6):
    df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
# Tách tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test_GradB, y_train, y_test_GradB = train_test_split(
    df[df.columns[:4]], df[df.columns[4]], test_size=0.25
)

# Tạo mô hình GBM ban đầu với tham số mặc định
model_GradB_ = GradientBoostingClassifier()

# Bộ cross-validation, với 5 fold, mỗi fold chứa 450 điểm dữ liệu
# Có sự xáo trộn, đồng thời 42 hạt ngẫu nhiên để đảm bảo tính nhất quán của kết quả
kf = KFold(n_splits= 5, shuffle=True, random_state=42)
# Tạo lưới tham số
param_grid = {
    "n_estimators": [100, 200, 300], # Số lượng cây quyết định 
    "learning_rate": [0.01, 0.05, 0.1], # Tốc độ học tập của thuật toán
    "max_depth": [5, 6, 7], # Chiều sâu tối đa của mỗi cây quyết định
    "subsample": [0.7, 0.8, 0.9], # Tỷ lệ lấy mẫu các điểm dữ liệu trong mỗi lần huấn luyện cây
}

# Thực hiện cross-validation để tìm ra bộ tham số tối ưu
# cross-validation với 30 lần, mỗi lần với một bộ tham số khác nhau
# Sau khi hoàn tất sẽ trả về bộ tham số cho mô hình có độ chính xác cao nhất trên tập kiểm tra
# Fine-tuning tinh chỉnh các tham số của mô hình để cải thiện hiệu suất của mô hình
grid_search_GradB = GridSearchCV(
    model_GradB_, param_grid, scoring="accuracy", cv=kf, n_jobs=-1
)
grid_search_GradB.fit(X_train, y_train)


# Huấn luyện mô hình với bộ tham số tối ưu, sau đó dự đoán trên tập test
model_GradB = GradientBoostingClassifier(**grid_search_GradB.best_params_)
model_GradB.fit(X_train, y_train)

# Dự đoán các điểm dữ liệu trong tập kiểm tra
y_pred_GradB = model_GradB.predict(X_test_GradB)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\2668784121.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\2668784121.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\2668784121.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead

In [14]:
print("=========================Đánh giá mô hình Gradient Boosting===========================")
print("Bộ tham số ban đầu:", 
      'learning_rate:', model_GradB_.get_params()['learning_rate'],
      ', max_depth:', model_GradB_.get_params()['max_depth'],
      ', n_estimators:', model_GradB_.get_params()['n_estimators'],
      ', subsample:', model_GradB_.get_params()['subsample'])
print('Bộ tham số tối ưu: ', grid_search_GradB.best_params_)
print(classification_report(y_test_GradB, y_pred_GradB))
print("Độ chính xác:", model_GradB.score(X_test_GradB, y_test_GradB))

=========================Đánh giá mô hình Gradient Boosting===========================
Bộ tham số ban đầu: learning_rate: 0.1 , max_depth: 3 , n_estimators: 100 , subsample: 1.0
Bộ tham số tối ưu:  {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}
              precision    recall  f1-score   support

           1       0.62      0.21      0.31       252
           2       0.70      0.93      0.80       498

    accuracy                           0.69       750
   macro avg       0.66      0.57      0.56       750
weighted avg       0.67      0.69      0.64       750

Độ chính xác: 0.6906666666666667


In [15]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

df = theft_predict_df.copy()
# Tách tập dữ liệu thành tập huấn luyện và tập kiểm tra
for i in range(1, 6):
    df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes 
X_train, X_test_XGB, y_train, y_test_XGB = train_test_split(
    df[df.columns[:4]], df[df.columns[4]], test_size=0.25
)
# Tạo mô hình XGBoost ban đầu với tham số mặc định
_model_XGB = XGBClassifier()

# Bộ cross-validation, với 5 fold, mỗi fold chứa 450 điểm dữ liệu
# Có sự xáo trộn, đồng thời 42 hạt ngẫu nhiên để đảm bảo tính nhất quán của kết quả
kf = KFold(n_splits= 5, shuffle=True, random_state=42)
# Tạo lưới tham số
param_grid = {
    "n_estimators": [50, 100, 200], # Số lượng cây quyết định 
    "learning_rate": [0.01, 0.05, 0.1], # Tốc độ học tập của thuật toán
    "max_depth": [5, 6, 7], # Chiều sâu tối đa của mỗi cây quyết định
    "subsample": [1.0, 0.9, 0.8], # Tỷ lệ lấy mẫu các điểm dữ liệu trong mỗi lần huấn luyện cây
}


# Thực hiện cross-validation để tìm ra bộ tham số tối ưu
# cross-validation với 30 lần, mỗi lần với một bộ tham số khác nhau
# Sau khi hoàn tất sẽ trả về bộ tham số cho mô hình có độ chính xác cao nhất trên tập kiểm tra
# Fine-tuning tinh chỉnh các tham số của mô hình để cải thiện hiệu suất của mô hình
grid_search_XGB = GridSearchCV(
    _model_XGB, param_grid, scoring="accuracy", cv=kf, n_jobs=-1
)
grid_search_XGB.fit(X_train, y_train)

# Huấn luyện mô hình
# Huấn luyện mô hình với bộ tham số tối ưu, sau đó dự đoán trên tập test
model_XGB = XGBClassifier(**grid_search_XGB.best_params_)
model_XGB.fit(X_train, y_train)

# Dự đoán các điểm dữ liệu trong tập kiểm tra
y_pred_XGB = model_XGB.predict(X_test_XGB)



C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\4183639006.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\4183639006.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\4183639006.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of alwa

In [16]:
print("=========================Đánh giá mô hình XGBoost===========================")
print("Bộ tham số ban đầu:", 
      'learning_rate:', _model_XGB.get_params()['learning_rate'],
      ', max_depth:', _model_XGB.get_params()['max_depth'],
      ', n_estimators:', _model_XGB.get_params()['n_estimators'],
      ', subsample:', _model_XGB.get_params()['subsample'])
print('Bộ tham số tối ưu: ', grid_search_XGB.best_params_)
print(classification_report(y_test_XGB, y_pred_XGB))
print("Độ chính xác:", model_XGB.score(X_test_XGB, y_test_XGB))

=========================Đánh giá mô hình XGBoost===========================
Bộ tham số ban đầu: learning_rate: None , max_depth: None , n_estimators: None , subsample: None
Bộ tham số tối ưu:  {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 50, 'subsample': 0.8}
              precision    recall  f1-score   support

           0       0.53      0.32      0.40       280
           1       0.67      0.83      0.74       470

    accuracy                           0.64       750
   macro avg       0.60      0.58      0.57       750
weighted avg       0.62      0.64      0.62       750

Độ chính xác: 0.6413333333333333


In [17]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

# Chuyển các cột string thành các cột số 
df = theft_predict_df.copy()
for i in range(1, 5):
    df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1

# Tách tập dữ liệu thành tập huấn luyện (chiếm 75%) và tập kiểm tra
X_train, X_test_LGBM, y_train, y_test_LGBM = train_test_split(
    df[df.columns[:4]], df[df.columns[4]], test_size=0.25
)
# Khai báo mô hình LightGBM
_model_LGBM = LGBMClassifier()


# Bộ cross-validation, với 5 fold, mỗi fold chứa 450 điểm dữ liệu
# Có sự xáo trộn, đồng thời 42 hạt ngẫu nhiên để đảm bảo tính nhất quán của kết quả
kf = KFold(n_splits= 5, shuffle=True, random_state=42)
# Tạo lưới tham số
param_grid = {
    "n_estimators": [100, 250, 400], # Số lượng cây quyết định 
    "learning_rate": [0.01, 0.05, 0.1], # Tốc độ học tập của thuật toán
    "max_depth": [4, 5, 6], # Chiều sâu tối đa của mỗi cây quyết định
    "min_child_samples": [10, 25, 40], # Số lượng mẫu tối thiểu cần thiết trong một nút lá.
    "subsample": [0.6, 0.75, 0.9], # Tỷ lệ lấy mẫu các điểm dữ liệu trong mỗi lần huấn luyện cây
}


# Thực hiện cross-validation để tìm ra bộ tham số tối ưu
# cross-validation với 30 lần, mỗi lần với một bộ tham số khác nhau
# Sau khi hoàn tất sẽ trả về bộ tham số cho mô hình có độ chính xác cao nhất trên tập kiểm tra
# Fine-tuning tinh chỉnh các tham số của mô hình để cải thiện hiệu suất của mô hình
grid_search_LGBM = GridSearchCV(
    _model_LGBM, param_grid, scoring="accuracy", cv=kf, n_jobs=-1
)
grid_search_LGBM.fit(X_train, y_train)


# Huấn luyện mô hình với bộ tham số tối ưu, sau đó dự đoán trên tập test
model_LGBM = LGBMClassifier(**grid_search_LGBM.best_params_, num_leaves= 15)
model_LGBM.fit(X_train, y_train)

y_pred_LGBM = model_LGBM.predict(X_test_LGBM)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\2074814121.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10668\2074814121.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f"col{i}"] = pd.Categorical(df.loc[:, f"col{i}"]).codes + 1
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for 

[LightGBM] [Info] Number of positive: 1459, number of negative: 791
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 122
[LightGBM] [Info] Number of data points in the train set: 2250, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.648444 -> initscore=0.612209
[LightGBM] [Info] Start training from score 0.612209
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [18]:
print("====================Đánh giá mô hình LGBM==================")
print("Bộ tham số ban đầu:", 
      'learning_rate:', _model_LGBM.get_params()['learning_rate'],
      ', max_depth:', _model_LGBM.get_params()['max_depth'],
      ', min_child_samples:',  _model_LGBM.get_params()['min_child_samples'],
      ', n_estimators:', _model_LGBM.get_params()['n_estimators'],
      ', subsample:', _model_LGBM.get_params()['subsample'])
print('Bộ tham số tối ưu: ', grid_search_LGBM.best_params_)
print(classification_report(y_test_LGBM, y_pred_LGBM))
print("Độ chính xác:", model_LGBM.score(X_test_LGBM, y_test_LGBM))


====================Đánh giá mô hình LGBM==================
Bộ tham số ban đầu: learning_rate: 0.1 , max_depth: -1 , min_child_samples: 20 , n_estimators: 100 , subsample: 1.0
Bộ tham số tối ưu:  {'learning_rate': 0.01, 'max_depth': 4, 'min_child_samples': 40, 'n_estimators': 250, 'subsample': 0.6}
              precision    recall  f1-score   support

        HIGH       0.65      0.26      0.37       278
         LOW       0.68      0.92      0.78       472

    accuracy                           0.67       750
   macro avg       0.66      0.59      0.57       750
weighted avg       0.67      0.67      0.63       750

Độ chính xác: 0.672


In [19]:
# TEST FOR LightGBM
print(model_LGBM.predict(np.array(X_test_LGBM.iloc[0]).reshape(-1,4))[0])
print(np.array(X_test_LGBM.iloc[0]))

HIGH
[72 47  4  2]


In [35]:
y_pred_LGBM_df = pd.DataFrame(y_pred_LGBM, columns=["prediction"])  # Tạo DataFrame với tên cột là "prediction"
df_new = pd.concat([X_test_LGBM, y_test_LGBM.to_frame().reset_index(drop=True), y_pred_LGBM_df], axis= 1) 
print(df_new.to_string())

     col1  col2  col3  col4  col5 prediction
0      72    47     4     2  HIGH       HIGH
1      11    30     3     1   LOW        LOW
2      11     3    10     2  HIGH        LOW
3      72    21     9     2   LOW        LOW
4      61    26     6     2   LOW        LOW
5      60    32     6     1   LOW        LOW
6      10    33    10     1   LOW        LOW
7      11    48    10     2   LOW        LOW
8      72    31    11     2   LOW        LOW
9      72     1    10     2  HIGH       HIGH
10     60     5     4     1   LOW        LOW
11     11     8    12     1   LOW        LOW
12     70     4     3     1   LOW       HIGH
13     72    17     2     2   LOW        LOW
14     72    37    10     1   LOW        LOW
15     11    42    12     1   LOW        LOW
16     45    37     8     2  HIGH        LOW
17     11    34    11     1   LOW        LOW
18     11     5    11     1   LOW        LOW
19     72    24    11     2   LOW        LOW
20     70    42     4     1  HIGH       HIGH
21     60 

- Khi không áp dụng Cross-validion và Fine-tuning, tốc độ của thuật toán nhanh, accuracy của mô hình rơi vào tầm 64 - 68%.
- Khi đã áp dụng Cross-validion và Fine-tuning, tốc độ của thuật toán chậm đi do phải điều chỉnh các tham số mô hình phù hợp, nhưng accuracy của mô hình đã được cải thiện hơn, dao động khoảng 65 - 70%.

# 5. Triển khai mô hình (model deployment)

Triển khai mô hình học máy (ML model deployment) là quá trình đưa mô hình học máy từ môi trường đào tạo sang môi trường thực tế để sử dụng, là một quá trình quan trọng để đưa mô hình từ lý thuyết sang thực tế. Quá trình này đòi hỏi sự hiểu biết về mô hình học máy, môi trường triển khai và các công cụ hỗ trợ.

Các tác dụng cụ thể của triển khai mô hình học máy là giúp tự động hóa các tác vụ thủ công hoặc tốn thời gian. Ví dụ, một mô hình học máy có thể được sử dụng để tự động dự đoán nguy cơ xảy ra tội phạm, dễ dàng phát hiện các dấu hiệu tội phạm bất thường.

In [21]:
pickle.dump(model_LGBM, open(str(os.getcwd()) + "\model.pkl", "wb"))

Một số công cụ triển khai mô hình máy học: Seldon.io, BentoML, TensorFlow Serving, Kubeflow, Cortex, AWS Sagemaker, Mlflow, Torchserve, Flask.

**Flask** là một microframework web Python nhẹ và linh hoạt, có thể được sử dụng để triển khai các mô hình học máy. Flask có một số ưu điểm khiến nó trở thành một lựa chọn tốt bởi vì:

- Có cú pháp đơn giản và dễ học, ngay cả đối với những người mới bắt đầu với Python.
- Linh hoạt, có thể được tùy chỉnh để đáp ứng nhu cầu cụ thể của ứng dụng.
- Hiệu suất cao,  hiệu quả về mặt tài nguyên, nên có vai trò quan trọng đối với việc triển khai các mô hình học máy lớn.
- Tích hợp với các mô hình học máy, có nhiều thư viện và công cụ có sẵn để giúp tích hợp các mô hình học máy với Flask.